# Run promptflow in cloud

In [ ]:
import os
import promptflow.azure as pf 
from promptflow.azure.entities import BulkFlowRunInput

# configure azureml workspace ml_client
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id="96aede12-2f73-41cb-b983-6d11a904839b",
    resource_group_name="promptflow",
    workspace_name="promptflow-canary",
)
pf.configure(client=client)

flow_dir = "."
data = "./data/denormalized-flat.jsonl"
runtime = "mir-0616"

In [ ]:
# load flow
flow = pf.load_flow(flow_dir)


In [ ]:
print(flow)

# Bulk Run in cloud

In [ ]:
# standard flow using remote data
run = flow.submit_bulk_run(
    data=data,
    connections={},
    runtime=runtime,
)
print(run)

In [ ]:
run.wait_for_completion()

In [ ]:
pf.show_details(run)

In [ ]:
pf.show_metrics(run)

# Eval run in cloud

In [ ]:
from promptflow.azure.entities import BulkFlowRunInput


classification_accuracy_eval = pf.load_flow("../classification_accuracy_evaluation")

bulk_flow_run_input = BulkFlowRunInput(
    data=data,
    variants=[run],
    inputs_mapping={"groundtruth": "data.intent", "prediction": "variants.output.output"},
)

baseline_accuracy = classification_accuracy_eval.submit_bulk_run(
    data=bulk_flow_run_input,
    runtime=runtime,
)

print(baseline_accuracy)

In [ ]:
baseline_accuracy.wait_for_completion()

In [ ]:
pf.show_details(baseline_accuracy)

In [ ]:
pf.show_metrics(baseline_accuracy)